## Java Memory Model
The JMM specifies the minimal guarantees the JVM must make about when writes to variables become visible to other threads. Modern shared-memory multiprocessors (and compilers) can do some surprising things when data is shared across threads.\

### Reordering
To make matters worse, the JMM can permit actions to appear to execute in different orders from the perspective of different threads, making reasoning about ordering in the absence of synchronization even more complicated. Consider the scenario below ($r1$, $r2$ are local variables, whereas $A$ and $B$ are shared):

<img src="images/reordering_1.png" >

It may appear that the result $r2 == 2, r1 == 1$ is impossible. Intuitively, either instruction 1 or instruction 3 should come first in an execution. If instruction 1 comes first, it should not be able to see the write at instruction 4. If instruction 3 comes first, it should not be able to see the write at instruction 2. However, compilers are allowed to reorder the instructions in either thread, when this does not affect the execution of that thread in isolation. Another example:

<img src="images/reordering_2.png" >

### Happens Before
JMM defines a partial ordering2 called happens-before on all actions within the program. To guarantee that the thread executing action B can see the
results of action A (whether or not A and B occur in different threads), there must be a *happens-before relationship* between A and B. Rules of happens-before:
- **Program order rule:** Each action in a thread happens-before every action in that thread that comes later in the program order.
- **Monitor lock rule:** An unlock on a monitor lock happens-before every subsequent lock on that same monitor lock.
- **Volatile variable rule:** A write to a volatile field happens-before every subsequent read of that same field.
- **Thread start rule:** A call to `Thread.start` on a thread happens-before every action in the started thread.
- **Thread termination rule:** Any action in a thread happens-before any other thread detects that thread has terminated, either by successfully
return from `Thread.join` or by `Thread.isAlive` returning `false`.
- **Interruption rule:** A thread calling `interrupt` on another thread happens-before the interrupted thread detects the interrupt (either by having `InterruptedException` thrown, or invoking `isInterrupted` or `interrupted`).
- **Finalizer rule:** The end of a constructor for an object happens-before the start of the finalizer for that object.
- **Transitivity:** If A happens-before B, and B happens-before C, then A happens-before C.

<img src="images/happens_before.png" >

## Thread Safety
Whenever more than one thread accesses a given state variable, and one of them might write to it, they all must coordinate their access to it using synchronization. Without proper synchronization, the program is broken and to fix it:
- Don’t share the state variable across threads;
- Make the state variable immutable; or
- Use synchronization whenever accessing the state variable.

>A class is thread-safe if it behaves correctly when accessed from multiple threads, regardless of the scheduling or interleaving of the execution of those threads by the runtime environment, and with no additional synchronization or other coordination on the part of the calling code.

Below is an example of thread safe class utilizing servlet:

In [ ]:
// Stateless objects are always thread-safe.
public class StatelessFactorizer implements Servlet {
    public void service(ServletRequest req, ServletResponse resp) {
        BigInteger i = extractFromRequest(req);
        BigInteger[] factors = factor(i);
        encodeIntoResponse(resp, factors);
    }
}

### Race Condition
Let's tweak the above example, to iinclude a hit counter:

In [ ]:
public class UnsafeCountingFactorizer implements Servlet {
    private long count = 0;
    public long getCount() { return count; }
    
    public void service(ServletRequest req, ServletResponse resp) {
        BigInteger i = extractFromRequest(req);
        BigInteger[] factors = factor(i);
        ++count;
        encodeIntoResponse(resp, factors);
    }
}

The `++count` operation is not atomic, it is made up of three smaller instructions - read value of count, add 1 to it and finally write updated value to the variable. This is a *read-modify-write* operation. In te previous example, if the counter is initially 9, with some unlucky timing each thread could read the value, see that it is 9, add one to it, and each set the counter to 10 - something that we don't want.

Above failure is an example of *race condition*. A race condition occurs when the correctness of a computation depends on the relative timing or interleaving of multiple threads by the runtime. In addition oto *read-modify-write* race condition, we can have *check-then-act* race condition. Below example of lazy initialization explains it:

In [ ]:
public class LazyInitRace {
    private ExpensiveObject instance = null;
    
    public ExpensiveObject getInstance() {
        if (instance == null)
            instance = new ExpensiveObject();
        return instance;
    }
}

The above example has race condition because two threads can view instance variable to be null at the same time, thereby two different instance objects get created, which isn't what we would want. To solve the problem, we need to perform the increment operation as an atomic operation.

In [ ]:
public class CountingFactorizer implements Servlet {
    private final AtomicLong count = new AtomicLong(0);
    public long getCount() { return count.get(); }
    
    public void service(ServletRequest req, ServletResponse resp) {
        BigInteger i = extractFromRequest(req);
        BigInteger[] factors = factor(i);
        count.incrementAndGet();
        encodeIntoResponse(resp, factors);
    }
}

### Locking
Just wrapping every shared state as thread-safe variable doesn't work. For example, consider the below example where we want to add caching to the factorization service:

In [ ]:
public class UnsafeCachingFactorizer implements Servlet {
    private final AtomicReference<BigInteger> lastNumber = new AtomicReference<BigInteger>();
    private final AtomicReference<BigInteger[]> lastFactors = new AtomicReference<BigInteger[]>();
    
    public void service(ServletRequest req, ServletResponse resp) {
        BigInteger i = extractFromRequest(req);
        if (i.equals(lastNumber.get()))
            encodeIntoResponse(resp, lastFactors.get());
        else {
            BigInteger[] factors = factor(i);
            lastNumber.set(i);
            lastFactors.set(factors);
            encodeIntoResponse(resp, factors);
        }
    }
}

Even though the atomic references are individually thread-safe, `UnsafeCachingFactorizer` has race conditions that could make it produce the wrong answer. When updating one, you must update the others in the same atomic operation. To solve the above problem, we can use synchronization block.

A synchronized block has two parts: a reference to an object that will serve as the lock, and a block of code to be guarded by that lock. A synchronized method is a shorthand for a synchronized block that spans an entire method body, and whose lock is the object on which the method is being invoked. (Static synchronized methods use the Class object for the lock.) Every Java object can implicitly act as a lock for purposes of synchronization;
these built-in locks are called intrinsic locks or monitor locks. When thread A attempts to acquire a lock held by thread B, A must wait, or block, until B releases it. If B never releases the lock, A waits forever.

Locks are **reentrant**, if a thread tries to acquire a lock that it already holds, the request succeeds. If threads were not reenterent, this would cause deadlock - imagine a thread calling another synchronized method of the same object from a synchronized method.

In [ ]:
// Thread-safe however it has big performance problem
public class SynchronizedFactorizer implements Servlet {
    private BigInteger lastNumber;
    private BigInteger[] lastFactors;
    
    public synchronized void service(ServletRequest req, ServletResponse resp) {
        BigInteger i = extractFromRequest(req);
        if (i.equals(lastNumber))
            encodeIntoResponse(resp, lastFactors);
        else {
            BigInteger[] factors = factor(i);
            lastNumber = i;
            lastFactors = factors;
            encodeIntoResponse(resp, factors);
        }
    }
}

Just wrapping the compound action with a synchronized block is not sufficient; if synchronization is used to coordinate access to a variable, it is needed everywhere that variable is accessed. This would mean any getters for the shared state must also be synchronized.

>For each mutable state variable that may be accessed by more than one thread, all accesses to that variable must be performed with the same lock held. In this case, we say that the variable is guarded by that lock.

Not all data needs to be guarded by locks—only mutable data that will be accessed from multiple threads. While synchronized methods can make indi
vidual operations atomic, additional locking is required when multiple operations are combined into a compound action. For example, consider the code block below, `Vector` has thread-safe methods, however the below code is not thread safe:

In [ ]:
// A check-then-act race condition
if (!vector.contains(element))
    vector.add(element);

The solution to poor performance, is to synchronize just the critical section:

In [ ]:
public class CachedFactorizer implements Servlet {
    private BigInteger lastNumber;
    private BigInteger[] lastFactors;
    private long hits;
    private long cacheHits;
    
    public synchronized long getHits() { return hits; }
    public synchronized double getCacheHitRatio() {
        return (double) cacheHits / (double) hits;
    }
    
    public void service(ServletRequest req, ServletResponse resp) {
        BigInteger i = extractFromRequest(req);
        BigInteger[] factors = null;
        synchronized (this) {
            ++hits;
            if (i.equals(lastNumber)) {
                ++cacheHits;
            factors = lastFactors.clone();
            }
        }
        
        if (factors == null) {
            factors = factor(i);
            synchronized (this) {
                lastNumber = i;
                lastFactors = factors.clone();
            }
        }
        encodeIntoResponse(resp, factors);
    }
}

Acquiring and releasing a lock has some overhead, so it is undesirable to break down synchronized blocks too far. Avoid holding locks during lengthy computations or operations at risk of not completing quickly such as network or console I/O. 

If an object has two independent state variables, it better to use to different lock objects such that two different threads can access two states concurrently.

In [ ]:
class SharedObject {
    private long c1 = 0;
    private long c2 = 0;
    private Object o1 = new Object();
    private Object o2 = new Object();

    public void incrementC1() {
        synchronized(o1){
            this.c1++;
        }
    }

    public void incrementC2() {
        synchronized(o2){
            this.c2++;
        }
    }
}

### Visibility
As discussed earlier, writes by a thread may not be visible to other threads. In the example below, `NoVisibility` could loop forever because the value of ready might never become visible to the reader thread. Even more strangely, `NoVisibility` could print zero because the write to ready might be made visible to the reader thread before the write to number, due to reordering:

In [ ]:
public class NoVisibility {
    private static boolean ready;
    private static int number;
    
    private static class ReaderThread extends Thread {
        public void run() {
            while (!ready)
                // Yield means the current thread is willing to relinquish its current use of processor
                // but it'd like to be scheduled back soon as possible.
                Thread.yield();
            System.out.println(number);
        }
    }
    
    public static void main(String[] args) {
        new ReaderThread().start();
        // The compiler can reorder the below two operations
        number = 42;
        ready = true;
    }
}

Usually lack of synchronization causes threads to receive stale data - however in one case the value can be totally random. Non volatile `long` and `double`  values (64 bit) are read as two 32 bit values.

**volatile:** ensure that updates to a variable are propagated predictably to other threads. Accessing a `volatile` variable performs no locking and so cannot cause the executing thread to block, making volatile variables a lighter-weight synchronization mechanism than `synchronized`.

When thread A writes to a volatile variable and subsequently thread B reads that same variable, the values of all variables that were visible to A prior to writing to the volatile variable become visible to B after reading the volatile variable.

Locking can guarantee both visibility and atomicity; volatile variables can only guarantee visibility. The semantics of volatile are not strong enough to make the increment operation volatile (`count++`) atomic, unless you can guarantee that the variable is written only from a single thread. We use volatile when the following conditions are met:
- Writes to the variable do not depend on its current value, or you can ensure that only a single thread ever updates the value;
- The variable does not participate in invariants with other state variables;
- Locking is not required for any other reason while the variable is being accessed.

The effect of the volatile keyword is approximately that each individual read or write operation on that variable is made atomically visible to all threads. Notably, however, an operation that requires more than one read/write, such as `i++`, which is equivalent to `i = i + 1`, which does one read and one write, is not atomic, since another thread may write to `i` between the read and the write.